### Cuss words! 
Sohier, who posted this dataset, mentioned that there was some profanity in these data. When I read that, I couldn't help but wonder just how much profanity there was...
<br><br>

Let's find out.
<br><br>

![Shit fuck damn!][1]


  [1]: http://static.fjcdn.com/gifs/Shit_2dde75_715622.gif

In [1]:
# import import import I love import
import numpy as np
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
# Read in CSV to dataframe
df = pd.read_csv('../input/hacker_news_sample.csv')

In [3]:
# Examine the fields
df.columns

We need to find all the text that contains cuss words. These are my three favorite cuss words: shit, fuck, damn, so we'll be searching for those. Anything more would be uncivilized. To do that, I'll need to search each row's "text" field for those cuss words and also keep track of the authors and the number of their posts which contain such nasty words.

In [ ]:
# Use regex to get the posts with cuss words
num = len(df["text"])
cuss_post = 0
authors = {}
for i in range(0,num):
    has_cuss = re.search(r"shit|fuck|damn", str(df["text"][i]))
    if has_cuss:
        cuss_post += 1
        author = str(df["by"][i])
        if author in authors.keys():
            authors[author] += 1
        else:
            authors[author] = 1

In [ ]:
print("Out of", num, "posts,", cuss_post, "posts have cuss words", '(' + str(round(cuss_post/num*100, 2)) + '%)')

In [ ]:
dff = pd.DataFrame.from_dict(authors, orient="index").reset_index()
dff.columns=['authors', 'num_posts']
dff.head(20)

This looks good. We're seeing some good cussing going on.  Now let's take a look at the top 10 cussers. Who has the most cuss-containing posts on Hacker News?

In [ ]:
dff.loc[dff["num_posts"].max()]
data = dff.sort_values("num_posts", ascending=False).head(10)
data

In [ ]:
viz = sns.barplot(y=data.authors, x=data.num_posts, orient="h", palette="Set1")
viz.set(xlabel="Number of posts with cuss words", ylabel="Author")
plt.show()

Evidently, **michaelochurch** has quite the potty mouth with over 300 posts to Hacker News containing cuss words. Well done!  

# Part 2:

From the comments, Matt wonders "How would the results change if it looked at the number of cuss words per post?"
<br><br>
Lets do a little more work and actually keep track of the cuss words in each post

In [ ]:
cuss_list = ["shit", "shitter", "fuck", "fucked", "fucking", "fucker", "damn", "damnit", "ass", "asshole"]

df_cuss = pd.DataFrame({"by": df.by, "text": df.text})
df_cuss.head()

Adding a new column,  a count of the cuss words in each post

In [ ]:
# This method isn't perfect. I should change this to use regex instead 
def count_cusses(text):
    cuss_list = ["shit", "shitter", "fuck", "fucked", "fucking", "fucker", "damn", "damnit", "ass", "asshole"]
    text = str(text)
    count = 0
    for word in text.split(" "):
        if word in cuss_list:
            count += 1
    return count

# Count the cusses in each post
df_cuss["total_cusses"] = df_cuss.text.apply(count_cusses)

# Only want posts with cuss words
df_cuss = df_cuss[df_cuss.total_cusses > 0]

In [ ]:
# Authors with the most cusses total
df_cuss.groupby("by").sum().sort_values("total_cusses", ascending=False)[:20]

In [ ]:
# Top 20 posts with the most cusses
df_cuss.sort_values("total_cusses", ascending=False)[:20]

In [ ]:
# Top 20 posts with the most cusses per word
def cusses_per_word(text):
    text = str(text)
    words_in_text = len(text.split(" "))
    num_cusses = count_cusses(text)
    return (num_cusses/words_in_text)

df_cuss["cusses_per_word"] = df_cuss.text.apply(cusses_per_word)

df_cuss.sort_values("cusses_per_word", ascending=False)[:20]

To do list:
* Change count_cusses() to use regex instead of looking for matches in cuss_list
* Get each authors average cuss/word ratio, show top 20